In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append('..')

import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
django.setup()

In [ ]:
from apps.gym import models
from apps.gym.selectors import (
    ExercisesSelector,
    ListExercisesLookups,
    MuscleGroupsSelector,
)
from apps.core.utils import BaseLookupDTO
from apps.core.constants import Language
from django.db.models import Prefetch

from django.db import connection
from django.db.models import F
from django.contrib.postgres.fields import jsonb
from django.http import JsonResponse

In [ ]:
muscle = models.MuscleGroups.objects.all()[0]
muscle

In [ ]:
qs = ExercisesSelector.list_exercises(
    language=Language.PT,
    lookups=ListExercisesLookups(
        primary_muscle=muscle,
    ),
)
qs

In [ ]:
muscles_qs = MuscleGroupsSelector.list_muscles(language=Language.PT)
muscles_qs

In [ ]:
models.Exercises.objects.annotate(name=F('name__pt')).values('name')

In [ ]:
qs = models.Exercises.objects.filter(name__pt='Supino')
for i in qs:
    print(i.name)
    print(i.pk)

In [ ]:
qs = models.Exercises.objects.values('id').annotate(name=F('name__pt')).values('name')
qs = qs.filter(name='Supino')
# qs.filter(name=None)
qs

In [ ]:
connection.queries

In [ ]:
from django.db.models import F, CharField
from django.db.models.expressions import ExpressionWrapper
from django.db.models.functions import Cast, JSONObject

models.Exercises.objects.annotate(
    portuguese_name=Cast(F('translations__pt__name'), output_field=CharField())
).values('id', 'portuguese_name')[0]['portuguese_name']

In [ ]:
qs = ExercisesSelector.list_exercises()
qs

In [ ]:
list(qs)

In [ ]:
JsonResponse(dict(a=list(qs)))

In [ ]:
class TestLookup(BaseLookupDTO):
    some: str
    value: str

In [ ]:
a = TestLookup(
    some=1,
    value=2,
)

In [1]:
from functools import reduce
from typing import Callable, Iterable, List, TypeVar

T = TypeVar('T')
U = TypeVar('U')

def custom_map(func: Callable[[T], U], iterable: Iterable[T]) -> List[U]:
    """
    Apply a function to every item of an iterable and return a list of the results.

    :param func: A function that takes an item of type T and returns an item of type U.
    :param iterable: An iterable of type T.
    :return: A list of items of type U after applying the function to each item in the iterable.
    """
    def reducer(acc: List[U], item: T) -> List[U]:
        return acc + [func(item)]

    return reduce(reducer, iterable, [])

# Example usage
result = custom_map(lambda x: x * 2, [1, 2, 3, 4, 5])
print(result)  # Output: [2, 4, 6, 8, 10]


[2, 4, 6, 8, 10]
